In [ ]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.12:3.3.1",
        "io.unitycatalog:unitycatalog-spark_2.12:0.2.0",
        "org.apache.hadoop:hadoop-aws:3.3.4",
        # "org.apache.hadoop:hadoop-common:3.3.4",
        "com.amazonaws:aws-java-sdk-bundle:1.12.262",
    ]

    def start(
        self,
        app_name: str = "LazySparkSession App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[1]",
    ):
        from pyspark.sql import SparkSession

        builder = (
            SparkSession.Builder().appName(app_name)
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            .config("spark.jars.packages", ",".join(self.packages))
            .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog")
            .config("spark.sql.catalog.unity", "io.unitycatalog.spark.UCSingleCatalog")
            .config("spark.sql.catalog.unity.uri", "http://127.0.0.1:8080")
            .config("spark.sql.catalog.unity.token", "")
            .config("spark.sql.defaultCatalog", "unity")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            # .config("spark.hadoop.fs.s3a.access.key", "guQHMBGAXnfsVlgfB6ro")
            # .config("spark.hadoop.fs.s3a.secret.key", "jlcXrCCxYDjkXttPrlPFLwb08l58v5tJflooKuQU")
            # .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:10000")
            .master(master_url)
        )

        return builder.getOrCreate()
    


# AWS_ACCESS_KEY="*****"
# AWS_SECRET_KEY="******"
# conf = (
#     SparkConf()
#     .set("spark.executor.extraJavaOptions","-Dcom.amazonaws.services.s3.enableV4=true")
#     .set("spark.driver.extraJavaOptions","-Dcom.amazonaws.services.s3.enableV4=true")
#     .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY)
#     .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_KEY)
# )

# scT=SparkContext(conf=conf)
# sql = SparkSession(scT)

In [ ]:
spark = LazySparkSession().start()

your 131072x1 screen size is bogus. expect trouble
25/06/04 19:09:32 WARN Utils: Your hostname, PC21-107632 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/04 19:09:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/elicacio/Workspace/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/elicacio/.ivy2/cache
The jars for the packages stored in: /home/elicacio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
io.unitycatalog#unitycatalog-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3b276223-d0c5-4957-984f-901c6ec25625;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.unitycatalog#unitycatalog-spark_2.12;0.2.0 in central
	found io.unitycatalog#unitycatalog-client;0.2.0 in central
	found org.slf4j#slf4j-api;2.0.13 in central
	found org.apache.logging.log4j#log4j-slf4j2-impl;2.23.1 in central
	found org.apache.logging.log4j#log4j-api;2.23.1 in central
	found org.apache.logging.log4j#log4j-core;2.23.1 in central
	found com.fasterxml.jackson.datatype#

25/06/04 19:10:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
# Show schemas (output = default)
spark.sql("SHOW SCHEMAS").show()

# Show tables
spark.sql("SHOW TABLES IN default").show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|        marksheet|      false|
|  default|marksheet_uniform|      false|
|  default|          numbers|      false|
|  default|   user_countries|      false|
+---------+-----------------+-----------+



In [4]:
spark.sql("SELECT * FROM default.marksheet LIMIT 5;").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`marksheet` doesn't exist.

In [5]:
# Create new schema
spark.sql("CREATE SCHEMA demo")

# Should now show two schemas: default and demo
spark.sql("SHOW SCHEMAS").show()

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [12]:
# Create a new table
spark.sql("""
CREATE TABLE demo.mytable (id INT, desc STRING)
USING delta
STORAGE_LOCATION 's3a://landing/tables/mytable'
""")
# Example location:
# LOCATION '/tmp/tables/mytable'

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'STORAGE_LOCATION'.(line 4, pos 0)

== SQL ==

CREATE TABLE demo.mytable (id INT, desc STRING)
USING delta
STORAGE_LOCATION 's3a://landing/tables/mytable'
^^^


In [11]:
spark.sql("DESCRIBE DETAIL demo.mytable").show()

AnalysisException: [DELTA_PATH_DOES_NOT_EXIST] file:/home/unitycatalog/s3a:/landing/tables/mytable doesn't exist, or is not a Delta table.;
DescribeDeltaDetailCommand
+- ResolvedTable io.unitycatalog.spark.UCSingleCatalog@7af9ecec, demo.mytable, DeltaTableV2(org.apache.spark.sql.SparkSession@202c19c2,file:/home/unitycatalog/s3a:/landing/tables/mytable,Some(CatalogTable(
Catalog: unity
Database: demo
Table: mytable
Created Time: Wed Jun 04 19:14:07 BRT 2025
Last Access: UNKNOWN
Created By: Spark 
Type: EXTERNAL
Provider: DELTA
Location: file:///home/unitycatalog/s3a:/landing/tables/mytable)),Some(demo.mytable),None,Map())


In [ ]:
# Insert new rows
spark.sql("INSERT INTO demo.mytable VALUES (1, 'test 1')")
spark.sql("INSERT INTO demo.mytable VALUES (2, 'test 2')")
spark.sql("INSERT INTO demo.mytable VALUES (3, 'test 3')")
spark.sql("INSERT INTO demo.mytable VALUES (4, 'test 4')")

# Read table
spark.sql("SELECT * FROM demo.mytable").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `demo`.`mytable` doesn't exist.;
AppendData RelationV2[] unity.demo.mytable unity.demo.mytable, false
+- LocalRelation [col1#184, col2#185]
